In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
!pip install --upgrade diffusers[torch] datasets invisible_watermark transformers accelerate safetensors torchmetrics Pillow


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


# Step 1: Generate Stable Diffusion Images

In [1]:
from pathlib import Path
import shutil
import uuid
from diffusers import DiffusionPipeline
import torch
import datasets
import pandas as pd

In [3]:
prompt_dataset = datasets.load_dataset("Gustavosta/Stable-Diffusion-Prompts")

Found cached dataset parquet (/home/daniel/.cache/huggingface/datasets/Gustavosta___parquet/Gustavosta--Stable-Diffusion-Prompts-eb720eed369c61bb/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-0.9", torch_dtype=torch.float16)
pipe.enable_model_cpu_offload()

rf_pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-0.9", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
rf_pipe.enable_model_cpu_offload()

In [ ]:
target_dir = Path("images")
if not target_dir.is_dir():
    target_dir.mkdir()
else:
    shutil.rmtree(target_dir)
    target_dir.mkdir()

prompts = []
generated_images = []
for prompt in prompt_dataset["train"]:
    try:
        prompt = prompt["Prompt"]
        image = pipe(prompt, output_type="latent").images

        image = rf_pipe(prompt=prompt, image=image).images[0]

        image_name = f"{str(uuid.uuid4())}.png"
        image_path = target_dir / image_name
        image.save(image_path)
        prompts.append(prompt)
        generated_images.append(str(image_path))

        df = pd.DataFrame(data={"image": generated_images, "prompt": prompts})
        df.to_json("sd_dataset.json", orient="records") # save this after every generation to not loose progress in case of crashing
    except:
        print("An error occured while generating image.")

# Step 2: Generate CLIP Scores for all the examples

In [1]:
import pandas as pd
from torchmetrics.multimodal.clip_score import CLIPScore
import PIL
import numpy as np
import torch
from tqdm import tqdm

In [2]:
metric = CLIPScore(model_name_or_path="openai/clip-vit-large-patch14") # openai/clip-vit-base-patch16

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [3]:
metric = metric.to("cuda")

In [4]:
df = pd.read_json("sd_dataset.json")

In [9]:
clip_scores = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        img = PIL.Image.open(row["image"])
        img = img.convert('RGB')
        np_img = np.array(img)
        clip_score = metric(torch.Tensor(np_img).to("cuda"), row["prompt"]).detach().cpu().numpy()
        img.close()
        clip_scores.append(clip_score)
    except Exception as e:
        print(e)
        clip_scores.append(np.nan)

  0%|                                               | 1/1431 [00:00<02:43,  8.76it/s]

The size of tensor a (101) must match the size of tensor b (77) at non-singleton dimension 1
The size of tensor a (81) must match the size of tensor b (77) at non-singleton dimension 1


  1%|▌                                             | 17/1431 [00:02<03:32,  6.65it/s]

The size of tensor a (96) must match the size of tensor b (77) at non-singleton dimension 1


  2%|▉                                             | 29/1431 [00:04<03:30,  6.68it/s]

...

100%|███████████████████████████████████████████▊| 1426/1431 [03:48<00:00,  6.98it/s]

The size of tensor a (83) must match the size of tensor b (77) at non-singleton dimension 1


100%|███████████████████████████████████████████▉| 1430/1431 [03:49<00:00,  7.72it/s]

The size of tensor a (82) must match the size of tensor b (77) at non-singleton dimension 1
The size of tensor a (84) must match the size of tensor b (77) at non-singleton dimension 1


100%|████████████████████████████████████████████| 1431/1431 [03:49<00:00,  6.24it/s]


In [10]:
scored_df = pd.concat((df, pd.DataFrame(data={"clip_score": clip_scores})), axis=1)

In [11]:
scored_df = scored_df.dropna()

In [13]:
scored_df["prompt"] = scored_df["prompt"].astype("str")

In [20]:
scored_df.to_json("sd_dataset_scored.json", orient="records")

# Step 3: Precompute CLIP embeddings for the image text pairs

In [51]:
from tqdm import tqdm
import pandas as pd
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

In [41]:
# Load data
df = pd.read_json("sd_dataset_scored.json")

In [42]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14", output_hidden_states=True).to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [52]:
clip_image_embeddings = []
clip_text_embeddings = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    with Image.open(row["image"]) as img:
        inputs = processor(text=[row["prompt"]], images=[img], return_tensors="pt", padding=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            image_embedding = outputs["image_embeds"].detach().cpu().numpy()[0]
            text_embedding = outputs["text_embeds"].detach().cpu().numpy()[0]
    clip_image_embeddings.append(image_embedding)
    clip_text_embeddings.append(text_embedding)

100%|███████████████████████████████████████| 1250/1250 [01:31<00:00, 13.70it/s]


In [53]:
df["clip_text_embedding"] = [e.tolist() for e in clip_text_embeddingsxt_embeddings]
df["clip_image_embedding"] = [e.tolist() for e in clip_image_embeddings]

In [54]:
df.to_json("sd_dataset_scored_embedded.json")

# Step 4: Selection of especially appealing images according to CLIP

In [1]:
import pandas as pd
import numpy as np
from sliceguard import SliceGuard
from renumics import spotlight
from renumics.spotlight import Image, Embedding

In [2]:
df = pd.read_json("sd_dataset_scored_embedded.json")

In [7]:
clip_text_embeddings = np.vstack(df["clip_text_embedding"])
clip_image_embeddings = np.vstack(df["clip_image_embedding"])

## Global selection

In [9]:
def return_precomputed_metric(y, y_pred):
    return y.mean(0)

In [27]:
sg = SliceGuard()
sg.find_issues(df, ["clip_image_embedding"],
               "clip_score",
               "clip_score",
               return_precomputed_metric,
               metric_mode="min",
               min_support=2,
               min_drop=6.0,
              precomputed_embeddings={"clip_image_embedding": clip_image_embeddings})

# Note: There is no explicit interface for using a precomputed metric, however just supply the metric column
# for y and y_pred and return y mean in your metric function

The overall metric value is 31.099600642394634
Using 2 as minimum support for determining problematic clusters.
Using 6.0 as minimum drop for determining problematic clusters.
Identified 1 problematic slices.


,issue,issue_metric,issue_explanation
0,-1,NaN,
1,-1,NaN,
2,-1,NaN,
3,-1,NaN,
4,-1,NaN,
...,...,...,...
1245,-1,NaN,
1246,-1,NaN,
1247,-1,NaN,
1248,-1,NaN,


In [28]:
sg.report(spotlight_dtype={"image": Image, "clip_image_embedding": Embedding})

,image,prompt,clip_score,clip_text_embedding,clip_image_embedding,issue,issue_metric,issue_explanation,sg_emb_clip_image_embedding
0,images/56477cc5-64d0-4ba6-971a-72c719460235.png,"steampunk market interior, colorful, 3 d scene...",26.963913,"[-0.039592452300000004, -0.0147153912, 0.02259...","[0.042762961200000005, -0.0210433938, 0.045792...",-1,NaN,,"[0.042762961200000005, -0.0210433938, 0.045792..."
1,images/bd09826a-796e-4142-862d-1dacdeb3c4f5.png,"“A portrait of a cyborg in a golden suit, D&D ...",29.205557,"[-0.0020829607000000003, 0.0192637891, -0.0362...","[0.018390106, -0.0414507277, 0.0406149887, -0....",-1,NaN,,"[0.018390106, -0.0414507277, 0.0406149887, -0...."
2,images/8fe45789-ed16-4260-bdad-048485ab90e4.png,A full portrait of a beautiful post apocalypti...,30.058372,"[0.0062153996, 0.021728647900000002, -0.046727...","[0.0483528748, -0.050396055, 0.034902714200000...",-1,NaN,,"[0.0483528748, -0.050396055, 0.034902714200000..."
3,images/aa0ac4e2-38f2-4ce7-8a2c-69890b3f7173.png,"beautiful victorian raven digital painting, ar...",33.178711,"[0.0280955508, -0.0245218761, -0.0008252721000...","[0.0324550271, -0.0018246276, 0.0379409231, -0...",-1,NaN,,"[0.0324550271, -0.0018246276, 0.0379409231, -0..."
4,images/fc8fdc82-9bd2-4015-b434-b6958f9f0d80.png,"ilya kuvshinov with long sky blue hair, gold e...",31.954140,"[0.032235581400000005, -0.0244684052, -0.04714...","[0.0431220941, -0.0332533978, 0.02237399850000...",-1,NaN,,"[0.0431220941, -0.0332533978, 0.02237399850000..."
...,...,...,...,...,...,...,...,...,...
1245,images/9e3b26ab-b737-472e-815f-7c13389dd2f0.png,amazing lifelike award winning pencil illustra...,32.023315,"[-0.0161065459, 0.0105979368, 0.040526025, -0....","[0.0670162812, 0.02718102, 0.0588702112, -0.04...",-1,NaN,,"[0.0670162812, 0.02718102, 0.0588702112, -0.04..."
1246,images/4c3264ab-da87-4ff5-93fb-0a12bb5ed64e.png,A castle made out of white stone burning from ...,26.541388,"[-0.0453475416, 0.0172001105, -0.0454900749000...","[-0.0132969152, -0.0348297916, -0.0372253507, ...",-1,NaN,,"[-0.0132969152, -0.0348297916, -0.0372253507, ..."
1247,images/bace5279-aba0-499c-8c78-f3dbaeae50f8.png,"You keep on creepin', let the night be scary",23.000858,"[-0.024687962600000002, 0.018290622200000002, ...","[0.0080016209, 0.0243713874, 0.0124144787, -0....",-1,NaN,,"[0.0080016209, 0.0243713874, 0.0124144787, -0...."
1248,images/3d58bdd1-f6e3-455a-b0ee-0db53b14a4f1.png,"photo, young female meth producer, meth lab, c...",24.982887,"[0.0028408652, 0.0373360664, -0.00086677050000...","[0.0246292036, 0.054845836, 0.0172184352, -0.0...",-1,NaN,,"[0.0246292036, 0.054845836, 0.0172184352, -0.0..."


## Category-wise, adaptive selection

In [30]:
from hnne import HNNE

In [31]:
hnne = HNNE(metric="euclidean")

**Visualization idea: Cluster map with clip score and images, animated over best slices probably stratified according to clustering**

# Detect Biases in Stable Diffusion and CLIP Score Metric